In [184]:

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from config import password


## Merge netflix and omdb csv files

In [185]:
file = "resources/netflix_titles.csv"
netflix_df = pd.read_csv(file)
file = "resources/omdb_list.csv"
omdb_df = pd.read_csv(file)

In [186]:
omdb_df.head()

,Unnamed: 0,title,genre,runtime,imdbRating,imdbVotes,poster,awards,boxoffice,language
0,0,Norm of the North: King Sized Adventure,"Animation, Adventure, Comedy, Family",90 min,3.3,311,https://m.media-amazon.com/images/M/MV5BNjMwZD...,NaN,NaN,English
1,1,Jandino: Whatever it Takes,Comedy,95 min,4.8,23,https://m.media-amazon.com/images/M/MV5BMWE3MG...,NaN,NaN,"English, Dutch"
2,2,Transformers Prime,"Animation, Action, Adventure, Comedy, Drama, F...",30 min,7.9,"5,454",https://m.media-amazon.com/images/M/MV5BMTczND...,14 wins & 26 nominations.,NaN,English
3,3,Transformers: Robots in Disguise,"Animation, Action, Adventure, Comedy, Sci-Fi",22 min,6.0,842,https://m.media-amazon.com/images/M/MV5BMjMwNT...,2 wins & 11 nominations.,NaN,English
4,4,Apaches,Drama,82 min,5.9,292,https://m.media-amazon.com/images/M/MV5BODYyOT...,3 nominations.,NaN,"French, Arabic"


In [187]:
omdb_unique_df = omdb_df.drop_duplicates(subset=['title'])
print(len(omdb_df))
print(len(omdb_unique_df))

5473
5412


In [188]:
merged_df = netflix_df.merge(omdb_unique_df, how="left", on="title")
merged_df['imdbVotes']= merged_df['imdbVotes'].str.replace(',', '')
merged_df['imdbVotes']= merged_df['imdbVotes'].fillna(0)
merged_df['imdbVotes']= merged_df['imdbVotes'].astype(int)
print(len(merged_df))
merged_df.head()

6234


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,...,description,Unnamed: 0,genre,runtime,imdbRating,imdbVotes,poster,awards,boxoffice,language
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,...,Before planning an awesome wedding for his gra...,0.0,"Animation, Adventure, Comedy, Family",90 min,3.3,311,https://m.media-amazon.com/images/M/MV5BNjMwZD...,NaN,NaN,English
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,...,Jandino Asporaat riffs on the challenges of ra...,1.0,Comedy,95 min,4.8,23,https://m.media-amazon.com/images/M/MV5BMWE3MG...,NaN,NaN,"English, Dutch"
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,...,"With the help of three human allies, the Autob...",2.0,"Animation, Action, Adventure, Comedy, Drama, F...",30 min,7.9,5454,https://m.media-amazon.com/images/M/MV5BMTczND...,14 wins & 26 nominations.,NaN,English
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,...,When a prison ship crash unleashes hundreds of...,3.0,"Animation, Action, Adventure, Comedy, Sci-Fi",22 min,6.0,842,https://m.media-amazon.com/images/M/MV5BMjMwNT...,2 wins & 11 nominations.,NaN,English
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,...,When nerdy high schooler Dani finally attracts...,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN


## Transform netflix genre

In [189]:
# Turn listed_in into list
merged_df.listed_in = merged_df.listed_in.str.split(',').tolist()
listed_df = merged_df.dropna(subset=['listed_in'])
listed_in = listed_df["listed_in"].tolist()

In [190]:
# Loop through listed_in (netflix genre) and get unique values into netflix_genre set
netflix_genre = set()

for movie in listed_in:
    for x in movie:
        netflix_genre.add(x)
print(netflix_genre)

netflix_genre_id = pd.DataFrame(netflix_genre, columns=["netflix_genre"])
netflix_genre_id = netflix_genre_id.reset_index()
netflix_genre_id = netflix_genre_id.rename(columns={'index': 'netflix_genre_no'})
netflix_genre_id.head(45)

{'British TV Shows', 'Classic Movies', 'Independent Movies', 'Spanish-Language TV Shows', 'TV Comedies', ' Romantic TV Shows', ' Romantic Movies', ' Docuseries', 'TV Horror', ' International TV Shows', ' Independent Movies', ' Documentaries', 'Anime Series', ' TV Sci-Fi & Fantasy', ' Horror Movies', 'Children & Family Movies', 'Action & Adventure', ' TV Dramas', 'Romantic TV Shows', ' Anime Features', ' Reality TV', ' Stand-Up Comedy', 'Comedies', 'Crime TV Shows', ' Spanish-Language TV Shows', 'Anime Features', 'TV Action & Adventure', 'Music & Musicals', ' Classic & Cult TV', 'Cult Movies', "Kids' TV", ' Teen TV Shows', ' TV Action & Adventure', ' Classic Movies', ' Children & Family Movies', " Kids' TV", ' TV Mysteries', 'Thrillers', ' LGBTQ Movies', 'Stand-Up Comedy', ' Science & Nature TV', ' Thrillers', ' Comedies', 'Classic & Cult TV', ' Faith & Spirituality', 'Horror Movies', ' Dramas', 'International TV Shows', ' Music & Musicals', 'Stand-Up Comedy & Talk Shows', ' TV Comedies

,netflix_genre_no,netflix_genre
0,0,British TV Shows
1,1,Classic Movies
2,2,Independent Movies
3,3,Spanish-Language TV Shows
4,4,TV Comedies
5,5,Romantic TV Shows
6,6,Romantic Movies
7,7,Docuseries
8,8,TV Horror
9,9,International TV Shows


In [191]:
# Break out genres (listed_in) with show_id
listed_in_df = merged_df.loc[:,['show_id', 'listed_in']]
listed_in_df = listed_in_df.explode('listed_in')
listed_in_df = listed_in_df.rename(columns={'listed_in': 'netflix_genre'})
listed_in_df.head()

,show_id,netflix_genre
0,81145628,Children & Family Movies
0,81145628,Comedies
1,80117401,Stand-Up Comedy
2,70234439,Kids' TV
3,80058654,Kids' TV


In [192]:
# merge tables to create show_id and genre_id table
netflix_genre_table = listed_in_df.merge(netflix_genre_id,how = "left", on="netflix_genre")
netflix_genre_table = netflix_genre_table.loc[:,['show_id','netflix_genre_no']]
netflix_genre_table.head()                                        

,show_id,netflix_genre_no
0,81145628,15
1,81145628,42
2,80117401,39
3,70234439,30
4,80058654,30


## Transform omdb genre

In [193]:
# Turn genre into list 
merged_df.genre = merged_df.genre.str.split(',').tolist()
genre_df = merged_df.dropna(subset=['genre'])
genre = genre_df["genre"].tolist()

In [194]:
# Loop through genre (omdb genre) and get unique values into omdb_genre set
omdb_genre = set()

for movie in genre:
    for y in movie:
        omdb_genre.add(y)
print(omdb_genre)

omdb_genre_id = pd.DataFrame(omdb_genre, columns=["omdb_genre"])
omdb_genre_id = omdb_genre_id.reset_index()
omdb_genre_id = omdb_genre_id.rename(columns={'index': 'omdb_genre_no'})
omdb_genre_id.head()

{' Thriller', ' Romance', ' Animation', ' Crime', ' Action', 'Western', ' Mystery', 'Mystery', ' Reality-TV', 'Short', 'War', 'Comedy', 'Talk-Show', ' Adventure', ' Music', ' Musical', 'Reality-TV', ' Sci-Fi', ' Sport', ' War', 'Fantasy', ' Horror', 'Adventure', 'History', 'Horror', ' Biography', ' Family', ' Comedy', ' Western', 'Thriller', ' Fantasy', 'Animation', 'Music', ' Film-Noir', 'Action', ' Drama', ' Short', 'Musical', ' Documentary', ' Talk-Show', 'Biography', ' Game-Show', 'News', 'Documentary', 'Crime', ' News', ' History', 'Family', 'Sci-Fi', 'Adult', 'Sport', 'Romance', 'Game-Show', 'Drama'}


,omdb_genre_no,omdb_genre
0,0,Thriller
1,1,Romance
2,2,Animation
3,3,Crime
4,4,Action


In [195]:
# Break out genres with show_id
genre_df = merged_df.loc[:,['show_id', 'genre']]
genre_df = genre_df.explode('genre')
genre_df = genre_df.rename(columns={'genre': 'omdb_genre'})
genre_df.dropna(inplace = True)
genre_df.head()

,show_id,omdb_genre
0,81145628,Animation
0,81145628,Adventure
0,81145628,Comedy
0,81145628,Family
1,80117401,Comedy


In [196]:
# merge tables to create show_id and genre_id table
omdb_genre_table = genre_df.merge(omdb_genre_id,how = "left", on="omdb_genre")
omdb_genre_table = omdb_genre_table.loc[:,['show_id','omdb_genre_no']]
omdb_genre_table.head()

,show_id,omdb_genre_no
0,81145628,31
1,81145628,13
2,81145628,27
3,81145628,26
4,80117401,11


## Transform language table

In [197]:
# Turn language into list 
merged_df.language = merged_df.language.str.split(',').tolist()
lan_df = merged_df.dropna(subset=['language'])
language = lan_df["language"].tolist()
language

[['English'],
 ['English', ' Dutch'],
 ['English'],
 ['English'],
 ['French', ' Arabic'],
 ['English'],
 ['Spanish'],
 ['English'],
 ['English'],
 ['English', ' Dutch', ' German'],
 ['Hindi', ' English'],
 ['English', ' French'],
 ['English'],
 ['English'],
 ['Spanish', ' English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English', ' Mandarin'],
 ['English'],
 ['French'],
 ['French'],
 ['Hindi'],
 ['Telugu'],
 ['English', ' Arabic'],
 ['English'],
 ['Hindi'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['Thai'],
 ['Thai'],
 ['English'],
 ['Thai'],
 ['Thai'],
 ['Thai'],
 ['English'],
 ['English'],
 ['English'],
 ['Urdu'],
 ['English'],
 ['English'],
 ['Tamil'],
 ['English', ' Swahili', ' Nama', ' Xhosa', ' Korean'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English'],
 ['English', ' Italian'],
 ['English'],
 ['English'],
 ['Engl

In [198]:
# Loop through language (omdb language) and get unique values into language_set set
language_set = set()

for row in language:
    for i in row:
        language_set.add(i)
print(language_set)

language_df = pd.DataFrame(language_set, columns=["language"])
language_df = language_df.reset_index()
language_df = language_df.rename(columns={'index': 'language_no'})
language_df.head()

{' Berber languages', 'Serbo-Croatian', ' Nepali', 'Sanskrit', ' Xhosa', ' Kazakh', 'Hungarian', 'Tamil', ' Quechua', 'Indonesian', 'Malay', 'Hindi', 'Assamese', 'Wolof', 'Croatian', 'Yoruba', 'Akan', ' Filipino', 'Italian', 'Malayalam', ' Japanese', 'Ukrainian Sign Language', 'Georgian', ' Akan', ' Armenian', 'Gujarati', ' Kriolu', 'Latin', 'Hokkien', ' Italian', ' Somali', 'Khmer', ' Irish', 'Cantonese', ' Quenya', ' Dari', 'Dutch', ' Yoruba', ' Czech', ' Amharic', ' Washoe', 'Persian', ' Hungarian', ' Kurdish', ' Hindi', 'Neapolitan', 'Finnish', ' Bhojpuri', 'Bengali', ' Slovenian', ' Russian', ' Mongolian', ' Marathi', ' Japanese Sign Language', 'Korean', 'Catalan', 'Flemish', 'Pushto', ' Awadhi', 'Spanish', ' Hokkien', ' Estonian', ' Vietnamese', ' Danish', ' Ukrainian', ' Gujarati', ' Indonesian', ' Korean', ' Urdu', ' Manipuri', ' Egyptian (Ancient)', 'Romanian', ' Chechen', 'Polish', ' Indian Sign Language', ' Afrikaans', 'Urdu', ' Nama', ' Swiss German', 'Russian', ' Yiddish',

,language_no,language
0,0,Berber languages
1,1,Serbo-Croatian
2,2,Nepali
3,3,Sanskrit
4,4,Xhosa


In [199]:
# Break out languages with show_id
language_all_df = merged_df.loc[:,['show_id', 'language']]
language_all_df = language_all_df.explode('language')
language_all_df.dropna(inplace = True)
language_all_df.head()

,show_id,language
0,81145628,English
1,80117401,English
1,80117401,Dutch
2,70234439,English
3,80058654,English


In [200]:
# merge tables to create show_id and language_no table
language_table = language_all_df.merge(language_df,how = "left", on="language")
language_table = language_table.loc[:,['show_id','language_no']]
language_table.head()

,show_id,language_no
0,81145628,190
1,80117401,190
2,80117401,204
3,70234439,190
4,80058654,190


In [201]:
title_df = merged_df.drop(columns=["listed_in","genre","language","Unnamed: 0"])
title_df_show = title_df.loc[title_df.show_id == 80057969]
title_df_show

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,description,runtime,imdbRating,imdbVotes,poster,awards,boxoffice
19,80057969,Movie,Love,Gaspar Noé,"Karl Glusman, Klara Kristin, Aomi Muyock, Ugo ...","France, Belgium","September 8, 2017",2015,NR,135 min,A man in an unsatisfying marriage recalls the ...,135 min,6.1,48168,https://m.media-amazon.com/images/M/MV5BMTQzND...,2 wins & 1 nomination.,"$249,083"


## Load data into postgres using sqlalchemy

In [205]:
# create connection to ETL_project_DB in postgres
# import psycopg2
engine = create_engine('postgresql://postgres:'+ password + '@localhost:5432/')
connection = engine.connect()


In [206]:
# check table names in database
engine.table_names()




['OMDB_language',
 'OMDB_title_language',
 'OMDB_genre',
 'OMDB_title_genre',
 'Title',
 'Netflix_Listed_in',
 'Netflix_title_Listed_in']

In [207]:
# load dataframes into postgres using pandas
title_df.to_sql(name='Title', con=engine, if_exists='append', index=False)

netflix_genre_id.to_sql(name='Netflix_Listed_in', con=engine, if_exists='append', index=False)
netflix_genre_table.to_sql(name='Netflix_title_Listed_in', con=engine, if_exists='append', index=False)

omdb_genre_id.to_sql(name='OMDB_genre', con=engine, if_exists='append', index=False)
omdb_genre_table.to_sql(name='OMDB_title_genre', con=engine, if_exists='append', index=False)

language_df.to_sql(name='OMDB_language', con=engine, if_exists='append', index=False)
language_table.to_sql(name='OMDB_title_language', con=engine, if_exists='append', index=False)

KeyboardInterrupt: 